# Lightning Tour

Introduces the main ways of using Saber.

### Table of contents

1. [Training](#Training)
    1. [Preparing the dataset](#Preparing-the-dataset)
    2. [Preparing the embeddings](#Preparing-the-embeddings)
    3. [Transfer learning](#Transfer-learning)
    4. [Multi-task learning](#Multi-task-learning)
2. [Saving and loading](#Saving-and-loading)
    1. [Saving a model](#Saving-a-model)
    2. [Loading a model](#Loading-a-model)
3. [Performing predictions](#Preforming-predictions)
4. [Visualizations](#Visualizations)

## Training

Import the required modules

In [ ]:
from saber.config import Config
from saber.sequence_processor import SequenceProcessor

### Preparing the dataset

Saber requires all datasets to be provided in a CoNLL format with a BIO tag scheme, e.g.

```
Selegiline	B-CHED
-	O
induced	O
postural	B-DISO
hypotension	I-DISO
...
```

In this format, the first column contains each token of an input sentence, the last column contains the tokens tag, all columns are seperated by tabs, and all sentences by a newline. We have collected some corpora in this format [here](https://github.com/BaderLab/Biomedical-Corpora) for convenience.

> Corpora in the **Standoff** format can be converted to **CoNLL** format using [this](https://github.com/spyysalo/standoff2conll) tool. Corpora in **PubTator** format can be converted to **Standoff** first using [this](https://github.com/spyysalo/pubtator) tool.

Saber infers the "training strategy" based on the structure of the dataset folder:

- To use k-fold cross-validation, simply provide a `train.*` file in your dataset folder.

E.g.
```
.
├── NCBI-Disease
│   └── train.tsv
```

- To use a train/valid/test strategy, provide `train.*` and `test.*` files in your dataset folder. Optionally, you can provide a `valid.*` file. If not provided, a random 10% of examples from `train.*` are used as the validation set.

E.g.
```
.
├── NCBI-Disease
│   ├── test.tsv
│   └── train.tsv
```

### Preparing the embeddings

When training new models, you can (and should) provide your own pre-trained word embeddings with the `pretrained_embeddings` argument (see below). Saber expects all word embeddings to be in the `word2vec` file format. [Pyysalo _et al_. 2013](https://pdfs.semanticscholar.org/e2f2/8568031e1902d4f8ee818261f0f2c20de6dd.pdf) provide word embeddings that work quite well in the biomedical domain, which can be downloaded [here](http://bio.nlplab.org). Alternatively, from the command line call:

```bash
$ mkdir saber/word_embeddings
$ cd saber/word_embeddings
# Note: this file is over 4GB
$ wget http://evexdb.org/pmresources/vec-space-models/wikipedia-pubmed-and-PMC-w2v.bin
```

> Note: you do not need to download pre-trained word embeddings if you only plan on using Saber's pre-trained models.

#### GloVe

To use [GloVe](https://nlp.stanford.edu/projects/glove/) embeddings, just convert them to the [word2vec](https://code.google.com/archive/p/word2vec/) format first, e.g.

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.txt'
word2vec_output_file = 'word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)

### Training a model from scratch

Create a `SequenceProcessor` object. This object coordinates training, annotation, saving and loading of models and datasets

In [ ]:
sp = SequenceProcessor()

If not specified, SequenceProcessor uses the default `config.ini`. Alternatively, you can write your own `config.ini` file and specify it as follows 

In [ ]:
sp = SequenceProcessor(Config('path/to/your/config.ini')) # you don't HAVE to name it 'config.ini'!

See [here](https://github.com/BaderLab/saber/blob/master/saber/config.ini) for the default `config.ini` file.

> Note that if you are calling Saber from the command line, you can provide any parameter in the `config` file as a flag, e.g. `--config_filepath`.

We can then load the dataset, which is specified by the `dataset_folder` parameter (see [Preparing the dataset](#Preparing-the-dataset)). If pre-trained token embeddings were specified with `pretrained_embeddings`, you should load them here also (see [Preparing the embeddings](#Preparing-the-embeddings))

In [ ]:
sp.load_dataset()
# uncomment to load pre-trained word embeddings
# sp.load_embeddings()

> Note, you can actually pass the filepath to the dataset straight to the `load_dataset()` method, e.g. `sp.load_dataset('path/to/NCBI-Disease')`. The same is true for the `load_embeddings()` method.

Lastly, we create the model we would like to use (specified by the `model_name` parameter)

In [ ]:
sp.create_model()

> Again, `model_name` can alternatively be passed directly to `create_model()`.

We are then ready to train:

In [ ]:
sp.fit()

### Transfer learning

Transfer learning is as easy as training, saving, loading, and then continuing training of a model. Here is an example

In [ ]:
# Create and train a model on GENIA corpus
sp = SequenceProcessor()
sp.load_dataset('path/to/datasets/GENIA')
sp.create_model()
sp.fit()
sp.save('pretrained_models/GENIA')

# Load that model
del sp
sp = SequenceProcessor()
sp.load('pretrained_models/GENIA')

# Use transfer learning to continue training on a new dataset
sp.load_dataset('path/to/datasets/CRAFT')
sp.fit()

> Note that there is currently no way to easily do this with the command line interface, but I am working on it!

### Multi-task learning

Multi-task learning is as easy as specifying multiple dataset paths, either in the `config` file, at the command line via the flag `--dataset_folder`, or as an argument to `load_dataset()`. The number of datasets is arbitrary.

Here is an example using the last method

In [ ]:
sp = SequenceProcessor()

# Simply pass multiple dataset paths to load_dataset to use multi-task learning. 
sp.load_dataset('path/to/datasets/NCBI-Disease', 'path/to/datasets/Linnaeus')

sp.create_model()
sp.fit()

## Saving and loading

In the following sections we introduce the saving and loading of models.

### Saving a model

Assuming the model has already been created (see above), we can easily save our model like so

In [ ]:
path_to_saved_model = 'path/to/pretrained_models/PRGE'

sp.save(path_to_saved_model)

> Currently, `sp.save()` will save the weights from the last training epoch by default.

### Loading a model

Lets illustrate loading a model with a new `SequenceProccesor` object

In [ ]:
# Delete our previous SequenceProccesor object (if it exists)
if 'sp' in locals(): del sp

# Create a new SequenceProccesor object
sp = SequenceProcessor()

# Load a previous model
sp.load(path_to_saved_model)

## Preforming predictions

### Library

If you are using Saber as a `python` library, you can perform predictions on raw text with the `annotate()` method. Passing the argument `jupyter=True` allows us to render the result directly in the notebook

In [ ]:
# Assuming sp is a `SequenceProcessor` object with a loaded (and trained) model
sp.annotate('Viral-mediated noisy gene expression reveals biphasic E2f1 response to MYC Gene expression.', jupyter=True)

### Web-service

Sabers web-service is invoked locally from the shell with

```
$ python -m saber.app
```

> You have to run this from your terminal! Not in the notebook.

See [here](https://baderlab.github.io/saber-api-docs/) for Sabers web-service API docs.

## Visualizations

_Note: This is less a feature and more a by-product of the fact that the model is implemented in [Keras](https://keras.io)._

We can easily create an image depiction our model. First, install the [graphviz graph library](http://www.graphviz.org/) and the [Python interface](https://pypi.python.org/pypi/graphviz). This is useful if you plan on modifying the architecture of the model.

> More info can be found [here](https://machinelearningmastery.com/visualize-deep-learning-neural-network-model-keras/).

In [ ]:
sp = SequenceProcessor()

# set this variable equal to your Keras model object.
model_ = sp.model.model[0]

We can either: create and save an image on our local machine,

In [ ]:
from keras.utils import plot_model
plot_model(model_, to_file='model.png', show_shapes=True, show_layer_names=True)

or, visualize it directly in the notebook

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model_, show_shapes=True, show_layer_names=True).create(prog='dot', format='svg'))